# Demo 1 - Comparing BCI pipelines with MOABB - Code based on this [tutorial](http://moabb.neurotechx.com/docs/auto_examples/advanced_examples/plot_statistical_analysis.html#sphx-glr-download-auto-examples-advanced-examples-plot-statistical-analysis-py) 

In [ ]:
import matplotlib.pyplot as plt
from mne.decoding import CSP
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

import moabb
from moabb import benchmark, set_log_level
from moabb.analysis.plotting import score_plot
import moabb.analysis.plotting as moabb_plt
from moabb.analysis.meta_analysis import (  # noqa: E501
    compute_dataset_statistics,
    find_significant_differences,
)
from moabb.datasets import BNCI2014_001, Zhou2016, Cho2017
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import LeftRightImagery

import seaborn as sns

moabb.set_log_level("info")
print(__doc__)


In [ ]:
# pipelines to be compared

pipelines = {}

param_svm = {"kernel": ("linear", "rbf"), "C": [0.1, 1, 10]}
step_csp_svm = [
    ("cov", Covariances(estimator="lwf")),
    ("csp", CSP(6)),
    ("optsvm", GridSearchCV(SVC(), param_svm, cv=3)),
]
pipelines["CSP+optSVM"] = Pipeline(steps=step_csp_svm)


step_csp_lda=[("csp", CSP(8)), ("LDA", LDA())]
pipelines["CSP+LDA"] = Pipeline(steps=step_csp_lda)

step_rg_lr =[ ("cov", Covariances()), ("RG", TangentSpace()), ("LR", LogisticRegression())]
pipelines["RG+LR"] = Pipeline(steps=step_rg_lr)

step_csp_lr =[("csp", CSP(8)), ("LR", LogisticRegression())]
pipelines["CSP+LR"] = Pipeline(steps=step_csp_lr)

step_rg_lda =[("cov", Covariances()), ("RG", TangentSpace()), ("LDA", LogisticRegression())]
pipelines["RG+LDA"] = Pipeline(steps=step_rg_lda)


In [ ]:
# evaluation on one dataset

paradigm = LeftRightImagery()
dataset = BNCI2014_001()
dataset.subject_list = dataset.subject_list[:4]
datasets = [dataset]
overwrite = True  # set to False if we want to use cached results
evaluation = CrossSessionEvaluation(
    paradigm=paradigm, datasets=datasets, suffix="stats", overwrite=overwrite
)

results = evaluation.process(pipelines)

In [ ]:
# plot comparison
fig = moabb_plt.score_plot(results)
plt.show()


In [ ]:
fig = moabb_plt.paired_plot(results, "CSP+LDA", "RG+LDA")
plt.show()



In [ ]:
# plot statistical comparison
stats = compute_dataset_statistics(results)
P, T = find_significant_differences(stats)

fig = moabb_plt.meta_analysis_plot(stats, "CSP+LDA", "RG+LDA")
plt.show()

In [ ]:
moabb_plt.summary_plot(P, T)
plt.show()

# Demo 2 - Benchmarking with different datasets

In [ ]:
paradigm = LeftRightImagery()
subj = [1, 2, 3]
datasets = [BNCI2014_001(), Zhou2016()]
for d in datasets:
    d.subject_list = subj

overwrite = True  # set to False if we want to use cached results
evaluation = CrossSessionEvaluation(
    paradigm=paradigm, datasets=datasets, suffix="stats", overwrite=overwrite
)

results = evaluation.process(pipelines)

In [ ]:

sns.catplot(
    data=results,
    x="session",
    y="score",
    hue="subject",
    col="dataset",
    kind="bar",
    palette="viridis",
)
plt.show()

In [ ]:
fig = moabb_plt.score_plot(results)
plt.show()

In [ ]:
stats = compute_dataset_statistics(results)
P, T = find_significant_differences(stats)
moabb_plt.summary_plot(P, T)
plt.show()


In [ ]:
fig = moabb_plt.meta_analysis_plot(stats, "CSP+LDA", "RG+LR")
plt.show()

# Demo 3 - Use of alternative features based on functional connectivity - Code based on this [tutorial](https://pyriemann.readthedocs.io/en/latest/auto_examples/motor-imagery/plot_ensemble_coherence.html)

In [ ]:
import matplotlib.pyplot as plt

from mne import Epochs, pick_types, events_from_annotations
from mne.io import concatenate_raws
from mne.io.edf import read_raw_edf
from mne.datasets import eegbci

import numpy as np
import pandas as pd
import seaborn as sns

from pyriemann.classification import FgMDM
from pyriemann.estimation import Covariances, Coherences
from pyriemann.spatialfilters import CSP
from pyriemann.tangentspace import TangentSpace

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC


from helpers.coherence_helpers import (
    NearestSPD,
    get_results,
)

## Connectivity transformer to compute FC estimations

In [ ]:
class Connectivities(TransformerMixin, BaseEstimator):
    """Getting connectivity features from epoch"""

    def __init__(self, method="ordinary", fmin=8, fmax=35, fs=None):
        self.method = method
        self.fmin = fmin
        self.fmax = fmax
        self.fs = fs

    def fit(self, X, y=None):
        self._coh = Coherences(
            coh=self.method,
            fmin=self.fmin,
            fmax=self.fmax,
            fs=self.fs,
        )
        return self

    def transform(self, X):
        X_coh = self._coh.fit_transform(X)
        X_con = np.mean(X_coh, axis=-1, keepdims=False)
        return X_con

## EEG data

In [ ]:
# avoid classification of evoked responses by using epochs that start 1s after cue onset.
tmin, tmax = 1.0, 2.0
event_id = dict(hands=2, feet=3)
subject = 7
runs = [4, 8]  # motor imagery: left vs right hand

raw_files = [
    read_raw_edf(f, preload=True) for f in eegbci.load_data(subject, runs)
]
raw = concatenate_raws(raw_files)

picks = pick_types(
    raw.info, meg=False, eeg=True, stim=False, eog=False, exclude="bads"
)
# subsample elecs
picks = picks[::2]

# Apply band-pass filter
raw.filter(7.0, 35.0, method="iir", picks=picks)

events, _ = events_from_annotations(raw, event_id=dict(T1=2, T2=3))

# Read epochs (train will be done only between 1 and 2s)
epochs = Epochs(
    raw,
    events,
    event_id,
    tmin,
    tmax,
    proj=True,
    picks=picks,
    baseline=None,
    preload=True,
    verbose=False,
)
labels = epochs.events[:, -1] - 2
fs = epochs.info["sfreq"]
X = 1e6 * epochs.get_data()

In [ ]:
# plot infos features
idx_left=np.where(labels==0)
idx_right=np.where(labels==1)

X_left=np.squeeze(X[idx_left,:,:])
X_right=np.squeeze(X[idx_right,:,:])

mat_connect_left=dict()
mat_connect_right=dict()

param_ft = {"fmin": fmin, "fmax": fmax, "fs": fs}
for sm in spectral_met:
    if sm == "cov":
        Y=Covariances(estimator="lwf").transform(X)
        mat_connect_left[sm]=Y[idx_left,:,:]
        mat_connect_right[sm]=Y[idx_right,:,:]
    else:
        Y=Coherences(coh=sm, fmin=fmin, fmax=fmax, fs=fs).transform(X)
        avg_freqBand=Y.mean(axis=3)
        mat_connect_left[sm]=np.mean(np.squeeze(avg_freqBand[idx_left,:,:]),0) # direct averaged over the trials
        mat_connect_right[sm]=np.mean(np.squeeze(avg_freqBand[idx_right,:,:]),0)
        
        
# plot FC
sm=spectral_met[2]

fig, axes = plt.subplots(1,2, figsize=[8, 3])
ch_names = [ch.replace('.', '') for ch in epochs.ch_names]

df = pd.DataFrame(data=mat_connect_left[sm], index=ch_names, columns=ch_names)
g = sns.heatmap(
                df, ax=axes[0], square=True, cbar=True, xticklabels=2, yticklabels=2)
g.set_title('Mean ' + sm + ' - left MI')

df = pd.DataFrame(data=mat_connect_right[sm], index=ch_names, columns=ch_names)
g = sns.heatmap(
                df, ax=axes[1], square=True, cbar=True, xticklabels=2, yticklabels=2)
g.set_title('Mean ' + sm + ' - right MI')

plt.xticks(rotation='vertical')
plt.yticks(rotation='horizontal')
plt.show()


fig, axes = plt.subplots(figsize=[8, 4])
ch_names = [ch.replace('.', '') for ch in epochs.ch_names]

df = pd.DataFrame(data=mat_connect_right[sm]-mat_connect_left[sm], index=ch_names, columns=ch_names)
g = sns.heatmap(
                df, ax=axes, square=True, cbar=True, xticklabels=2, yticklabels=2, cmap="vlag")
g.set_title('Difference Right - Left ' + sm )

plt.xticks(rotation='vertical')
plt.yticks(rotation='horizontal')
plt.show()

## Pipelines definition

Here we are going to compare different methods: CSP+shLDA, CSP+SVM, FgMDM on covariance, tangent space logistic regression with covariance, lag coherence, and instantaneous coherence, along with ensemble method - *TODO: add more?*

In [ ]:
ppl_baseline, ppl_fc, ppl_ens = {}, {}, {}

In [ ]:
# Baseline pipelines
param_svm = {"kernel": ("linear", "rbf"), "C": [0.1, 1, 10]}
step_csp_svm = [
    ("cov", Covariances(estimator="lwf")),
    ("csp", CSP(nfilter=6)),
    ("optsvm", GridSearchCV(SVC(), param_svm, cv=3)),
]
ppl_baseline["CSP+optSVM"] = Pipeline(steps=step_csp_svm)


step_mdm = [
    ("cov", Covariances(estimator="lwf")),
    ("fgmdm", FgMDM(metric="riemann", tsupdate=False)),
]
ppl_baseline["FgMDM"] = Pipeline(steps=step_mdm)


In [ ]:
# Functional connectivity pipelines
spectral_met = ["cov", "lagged", "instantaneous"]
fmin, fmax = 8, 35
param_lr = {
    "penalty": "elasticnet",
    "l1_ratio": 0.15,
    "intercept_scaling": 1000.0,
    "solver": "saga",
}
param_ft = {"fmin": fmin, "fmax": fmax, "fs": fs}
step_fc = [
    ("spd", NearestSPD()),
    ("tg", TangentSpace(metric="riemann")),
    ("LogistReg", LogisticRegression(**param_lr)),
]
for sm in spectral_met:
    pname = sm + "+elasticnet"
    if sm == "cov":
        ppl_fc[pname] = Pipeline(
            steps=[("cov", Covariances(estimator="lwf"))] + step_fc
        )
    else:
        ft = Connectivities(**param_ft, method=sm)
        ppl_fc[pname] = Pipeline(steps=[("ft", ft)] + step_fc)

In [ ]:
# Ensemble classifier
fc_estim = [(n, ppl_fc[n]) for n in ppl_fc]
cvkf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

lr = LogisticRegression(**param_lr)
ppl_ens["ensemble"] = StackingClassifier(
    estimators=fc_estim,
    cv=cvkf,
    n_jobs=1,
    final_estimator=lr,
    stack_method="predict_proba",
)

### Evaluation

In [ ]:
dataset_res = list()
all_ppl = {**ppl_baseline, **ppl_ens}
#print(all_ppl)

In [ ]:
# Compute results
results = get_results(X, labels, all_ppl)
results = pd.DataFrame(results)

In [ ]:
results

In [ ]:
# plot results
list_fc_ens = ["ensemble", "CSP+optSVM", "FgMDM"] + \
    [sm + "+elasticnet" for sm in spectral_met]

g = sns.catplot(
    data=results,
    x="pipeline",
    y="score",
    kind="bar",
    order=list_fc_ens,
    height=7,
    aspect=2,
)
plt.show()